In [1]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker

import torch
from functions import *
import scib_metrics
%matplotlib inline

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [2]:
adata = sc.read(
    "data/human_pancreas_norm_complexBatch.h5ad"
)

In [3]:
adata

AnnData object with n_obs × n_vars = 16382 × 19093
    obs: 'tech', 'celltype', 'size_factors'
    layers: 'counts'

In [4]:
scanoramaPredict(adata,batch_label="tech")

Found 19093 genes among all datasets
[[0.         0.60656455 0.03605016 0.14971606 0.00348028 0.00099602
  0.00398406 0.01494024 0.06175299]
 [0.         0.         0.29780564 0.00656455 0.         0.
  0.00153492 0.         0.37899344]
 [0.         0.         0.         0.         0.         0.
  0.         0.38714734 0.12068966]
 [0.         0.         0.         0.         0.49187935 0.67320599
  0.48503454 0.00154879 0.00208855]
 [0.         0.         0.         0.         0.         0.46403712
  0.68851508 0.         0.00459482]
 [0.         0.         0.         0.         0.         0.
  0.45203377 0.00554785 0.00125313]
 [0.         0.         0.         0.         0.         0.
  0.         0.04451266 0.02916347]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.47117962]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]]
Processing datasets (4, 6)
Processing datasets (3, 5)
Processing datasets 

In [5]:
sdnormalAdata, vaeSD = trainModelBenchmark(adata.copy(), "sdnormal", max_epochs=50, batch_key="tech")

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 50/50: 100%|██████████| 50/50 [01:11<00:00,  1.45s/it, v_num=1, train_loss_step=1.33e+4, train_loss_epoch=1.34e+4]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [01:11<00:00,  1.44s/it, v_num=1, train_loss_step=1.33e+4, train_loss_epoch=1.34e+4]


In [6]:
mogAdata, vaeMG = trainModelBenchmark(adata.copy(), "mixofgaus", max_epochs=50, batch_key="tech")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 50/50: 100%|██████████| 50/50 [01:13<00:00,  1.41s/it, v_num=1, train_loss_step=1.1e+4, train_loss_epoch=1.35e+4] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [01:13<00:00,  1.48s/it, v_num=1, train_loss_step=1.1e+4, train_loss_epoch=1.35e+4]


In [7]:
vampAdata, vaeVP = trainModelBenchmark(adata.copy(), "vamp", max_epochs=50, batch_key="tech")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 50/50: 100%|██████████| 50/50 [01:26<00:00,  1.86s/it, v_num=1, train_loss_step=8.83e+3, train_loss_epoch=1.34e+4]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [01:26<00:00,  1.74s/it, v_num=1, train_loss_step=8.83e+3, train_loss_epoch=1.34e+4]


In [8]:
allAdata = adata.copy()
allAdata.obsm["scVISD"] = sdnormalAdata.obsm["scVI"]
allAdata.obsm["scVIMG"] = mogAdata.obsm["scVI"]
allAdata.obsm["scVIVAMP"] = vampAdata.obsm["scVI"]

: 

: 

In [ ]:
plotBenchmarkResults(allAdata,keys = ["Scanorama","scVISD","scVIMG","scVIVAMP"],label_key="celltype",batch_key="tech")